In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/raw-agents/all_agents_combined.csv


In [2]:
import os, re, ast, json, math, logging, numpy as np, pandas as pd
from pathlib import Path
SEED = 42

import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

# Show full content of each column
pd.set_option("display.max_colwidth", None)

# Show all columns (no "..." truncation)
pd.set_option("display.max_columns", None)

In [3]:
df= pd.read_csv("/kaggle/input/raw-agents/all_agents_combined.csv")

In [4]:
df.head()

,state,agentId,partner,name,givenName,email,slug,photoUrl,phoneNumber,adminOnlyPhoneNumber,servesOffers,servesListings,starRating,numReviews,pastYearDealsInRegion,pastYearDeals,veteran,isPremier,languages,complianceText,homeTransactionsLifetime,transactionVolumeLifetime,officeState,jobTitle,primaryServiceRegions,propertyTypes,dealPrices,agentType,businessMarket,businessMarketId,servicing,brokerageName,firstName,lastName,fullName,gender,hasPhotos,noPhoto,profileUrl,photoUrl640x460,profileContactEnabled,totalDealsInPastYear,isActive,isPartnerTeam,customJobTitle,licenseClassTitle,licenseNumber
0,Alabama,24746,False,David Squires,David,david.squires@redfin.com,david-squires,https://ssl.cdn-redfin.com/system_files/images/24746/640x460/6_63.jpg,(239) 341-5997,False,True,True,4.9,51.0,0,19.0,False,True,[],NaN,117.0,52659458.0,FL,Senior Agent,"['N Fort Myers', 'Lehigh Acres', 'Fort Myers', 'Fort Myers Beach', 'Bonita Springs-Estero', 'East Fort Myers']","['Single Family Residential', 'Condo/Co-op', 'Vacant Land', 'Mobile/Manufactured Home', 'Multi-Family (2-4 Unit)']","[499000, 435000, 245000, 355000, 170000, 265000, 1850000, 380000, 210000, 560000, 375000, 505000, 300000, 345000, 230000, 575000, 355000, 274000, 623500, 1000000, 2900000, 601000, 510000, 150000, 395000, 349999, 860000, 475000, 416000, 226000, 375000, 364000, 417500, 510000, 585000, 358999, 440000, 475000, 601000, 534299, 320000, 302000, 379000, 654500, 379000, 221498, 257054, 235000, 530000, 425000, 580000, 531840, 330000, 522570, 436000, 315000, 455000, 305000, 345000, 270000, 259000, 830000, 580000, 714900, 140000, 309999, 380000, 400000, 1100000, 475000, 770000, 275000, 457000, 320000, 270000, 770000, 340000, 225000, 289900, 422000, 460000, 350000, 245000, 555000, 242000, 664000]",1,Fort Myers,51,Fort Myers,Redfin,David,Squires,David Squires,2,True,False,/real-estate-agents/david-squires,https://ssl.cdn-redfin.com/system_files/images/24746/640x460/6_63.jpg,False,19.0,True,False,NaN,NaN,NaN
1,Alabama,37611,True,The Anderson Team - Fort Myers - Partner Team,The Anderson Team - Fort Myers,partner-team-1742577788@redfintest.com,the-anderson-team-fort-myers,https://ssl.cdn-redfin.com/system_files/images/37611/500x500/8_58.jpg,NaN,False,True,True,5.0,18.0,1,8.0,False,False,[],NaN,NaN,NaN,FL,Senior Agent,"['Cape Coral', 'N Fort Myers', 'Lehigh Acres', 'Fort Myers', 'East Fort Myers', 'Fort Myers Islands']","['Single Family Residential', 'Vacant Land', 'Townhouse', 'Condo/Co-op']","[280000, 33500, 457000, 17900, 363000, 129000, 298000, 4125, 349990, 10000, 230000, 290000, 25000, 348000, 26000, 15000, 15000, 26000, 34998, 18000, 16500, 18000, 18000, 19900, 14900, 60000, 15500, 152000, 145000, 269000, 195000, 110000, 140000, 10000, 200000, 158000, 259900, 17000, 198000, 30000, 14000, 11000, 278000, 216500, 33500, 13000, 55000, 55000, 585000, 349900, 12500, 394900, 25000, 22900, 14500, 13500, 325000, 19000, 19000, 21500, 28000, 19500, 80000]",2,Fort Myers,51,Fort Myers,Sellstate 5 Star Realty,The Anderson Team - Fort Myers,- Partner Team,The Anderson Team - Fort Myers - Partner Team,4,True,False,/real-estate-agents/the-anderson-team-fort-myers,NaN,True,8.0,True,False,NaN,NaN,NaN
2,Alabama,21113,True,Kathy Stobb,Kathy,kathy@eventiderealty.com,katherine-stobb,https://ssl.cdn-redfin.com/v594.0.0/images/serviceProvider/person-500x500.jpg,NaN,False,True,True,5.0,36.0,0,12.0,False,False,[],NaN,NaN,NaN,FL,Senior Agent,"['Cape Coral', 'Lehigh Acres', 'FL - Alva & LaBelle']","['Vacant Land', 'Single Family Residential', 'Condo/Co-op', 'Mobile/Manufactured Home', 'Multi-Family (2-4 Unit)', 'Multi-Family (5+ Unit)', 'Other']","[33000, 215000, 365000, 290000, 50000, 189900, 340000, 211500, 248000, 362500, 102500, 455000, 266000, 150000, 146000, 280000, 120000, 139000, 205500, 167000, 284900, 230000, 625000, 150000, 237000, 385000, 264000, 316819, 216000, 425000, 2100000, 515000, 631000, 275000, 463000, 435000, 204000, 237497, 25000, 155000, 269900, 335000, 284990, 435000, 290000, 65000, 4

In [5]:
df.shape

(356, 47)

In [6]:
for col in df.columns:
    print(col)

state
agentId
partner
name
givenName
email
slug
photoUrl
phoneNumber
adminOnlyPhoneNumber
servesOffers
servesListings
starRating
numReviews
pastYearDealsInRegion
pastYearDeals
veteran
isPremier
languages
complianceText
homeTransactionsLifetime
transactionVolumeLifetime
officeState
jobTitle
primaryServiceRegions
propertyTypes
dealPrices
agentType
businessMarket
businessMarketId
servicing
brokerageName
firstName
lastName
fullName
gender
hasPhotos
noPhoto
profileUrl
photoUrl640x460
profileContactEnabled
totalDealsInPastYear
isActive
isPartnerTeam
customJobTitle
licenseClassTitle
licenseNumber


In [7]:
df.isnull().sum()

state                          0
agentId                        0
partner                        0
name                           0
givenName                      0
email                          0
slug                           0
photoUrl                       3
phoneNumber                  190
adminOnlyPhoneNumber           0
servesOffers                   0
servesListings                 0
starRating                    88
numReviews                    84
pastYearDealsInRegion          0
pastYearDeals                 89
veteran                        0
isPremier                      0
languages                      0
complianceText               321
homeTransactionsLifetime     202
transactionVolumeLifetime    200
officeState                   74
jobTitle                       0
primaryServiceRegions          0
propertyTypes                  0
dealPrices                     0
agentType                      0
businessMarket                 0
businessMarketId               0
servicing 

In [8]:
def parse_jsonish(x):
    if isinstance(x, (list, dict)) or pd.isna(x): return x
    s = str(x).strip()
    if not s or s.lower() in {"nan","none","null"}: return np.nan
    try:
        return ast.literal_eval(s)      # handles Python-literal style
    except Exception:
        s2 = re.sub(r",\s*([}\]])", r"\1", s.replace("'", '"'))  # fix quotes & trailing commas
        try: return json.loads(s2)
        except Exception: return x

def ensure_list(v):
    if isinstance(v, list): return v
    if isinstance(v, str):  return [v]
    return [] if (pd.isna(v) or v is None) else [v]

def to_numeric(df, cols):
    for c in cols:
        if c in df.columns: df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


In [9]:
LIST_COLS = ["primaryServiceRegions", "propertyTypes", "dealPrices", "languages"]
for c in LIST_COLS:
    if c in df.columns:
        df[c] = df[c].apply(parse_jsonish).apply(ensure_list)

In [10]:
NUM_COLS = [
    "starRating","numReviews","pastYearDeals","pastYearDealsInRegion",
    "homeTransactionsLifetime","transactionVolumeLifetime"
]
df = to_numeric(df, NUM_COLS)

In [11]:
def clean_prices(seq):
    if not isinstance(seq, list): return []
    out = []
    for v in seq:
        try:
            fv = float(v)
            if fv > 0 and math.isfinite(fv): out.append(fv)
        except Exception:
            pass
    return out

if "dealPrices" in df.columns:
    prices = df["dealPrices"].apply(clean_prices)
    df["dealPrices_count"]  = prices.apply(len)
    df["dealPrices_median"] = prices.apply(lambda s: float(np.median(s)) if s else 0.0)
    df["dealPrices_q25"]    = prices.apply(lambda s: float(np.percentile(s,25)) if s else 0.0)
    df["dealPrices_q75"]    = prices.apply(lambda s: float(np.percentile(s,75)) if s else 0.0)
    df["dealPrices_min"]    = prices.apply(lambda s: float(np.min(s)) if s else 0.0)
    df["dealPrices_max"]    = prices.apply(lambda s: float(np.max(s)) if s else 0.0)
    df["dealPrices_std"]    = prices.apply(lambda s: float(np.std(s)) if len(s)>1 else 0.0)
else:
    for c in ["dealPrices_count","dealPrices_median","dealPrices_q25","dealPrices_q75",
              "dealPrices_min","dealPrices_max","dealPrices_std"]:
        df[c] = 0.0


In [12]:
BOOLS = ["partner","isPremier","servesOffers","servesListings","isActive","profileContactEnabled"]
for c in BOOLS:
    if c in df.columns:
        df[c] = df[c].map({True:True, False:False, "True":True, "False":False, 1:True, 0:False}).fillna(False).astype(bool)

df["num_service_regions"] = df.get("primaryServiceRegions", pd.Series([[]]*len(df))).apply(lambda x: len(x) if isinstance(x,list) else 0)
df["num_property_types"]  = df.get("propertyTypes", pd.Series([[]]*len(df))).apply(lambda x: len(x) if isinstance(x,list) else 0)

if "starRating" in df.columns:
    df["starRating"] = df["starRating"].clip(0,5).fillna(0.0)
if "numReviews" in df.columns:
    df["numReviews"] = df["numReviews"].fillna(0).astype(int)

In [13]:
df.head(3)

,state,agentId,partner,name,givenName,email,slug,photoUrl,phoneNumber,adminOnlyPhoneNumber,servesOffers,servesListings,starRating,numReviews,pastYearDealsInRegion,pastYearDeals,veteran,isPremier,languages,complianceText,homeTransactionsLifetime,transactionVolumeLifetime,officeState,jobTitle,primaryServiceRegions,propertyTypes,dealPrices,agentType,businessMarket,businessMarketId,servicing,brokerageName,firstName,lastName,fullName,gender,hasPhotos,noPhoto,profileUrl,photoUrl640x460,profileContactEnabled,totalDealsInPastYear,isActive,isPartnerTeam,customJobTitle,licenseClassTitle,licenseNumber,dealPrices_count,dealPrices_median,dealPrices_q25,dealPrices_q75,dealPrices_min,dealPrices_max,dealPrices_std,num_service_regions,num_property_types
0,Alabama,24746,False,David Squires,David,david.squires@redfin.com,david-squires,https://ssl.cdn-redfin.com/system_files/images/24746/640x460/6_63.jpg,(239) 341-5997,False,True,True,4.9,51,0,19.0,False,True,[],NaN,117.0,52659458.0,FL,Senior Agent,"[N Fort Myers, Lehigh Acres, Fort Myers, Fort Myers Beach, Bonita Springs-Estero, East Fort Myers]","[Single Family Residential, Condo/Co-op, Vacant Land, Mobile/Manufactured Home, Multi-Family (2-4 Unit)]","[499000, 435000, 245000, 355000, 170000, 265000, 1850000, 380000, 210000, 560000, 375000, 505000, 300000, 345000, 230000, 575000, 355000, 274000, 623500, 1000000, 2900000, 601000, 510000, 150000, 395000, 349999, 860000, 475000, 416000, 226000, 375000, 364000, 417500, 510000, 585000, 358999, 440000, 475000, 601000, 534299, 320000, 302000, 379000, 654500, 379000, 221498, 257054, 235000, 530000, 425000, 580000, 531840, 330000, 522570, 436000, 315000, 455000, 305000, 345000, 270000, 259000, 830000, 580000, 714900, 140000, 309999, 380000, 400000, 1100000, 475000, 770000, 275000, 457000, 320000, 270000, 770000, 340000, 225000, 289900, 422000, 460000, 350000, 245000, 555000, 242000, 664000]",1,Fort Myers,51,Fort Myers,Redfin,David,Squires,David Squires,2,True,False,/real-estate-agents/david-squires,https://ssl.cdn-redfin.com/system_files/images/24746/640x460/6_63.jpg,False,19.0,True,False,NaN,NaN,NaN,86,387500.0,302750.0,531380.0,140000.0,2900000.0,353690.279493,6,5
1,Alabama,37611,True,The Anderson Team - Fort Myers - Partner Team,The Anderson Team - Fort Myers,partner-team-1742577788@redfintest.com,the-anderson-team-fort-myers,https://ssl.cdn-redfin.com/system_files/images/37611/500x500/8_58.jpg,NaN,False,True,True,5.0,18,1,8.0,False,False,[],NaN,NaN,NaN,FL,Senior Agent,"[Cape Coral, N Fort Myers, Lehigh Acres, Fort Myers, East Fort Myers, Fort Myers Islands]","[Single Family Residential, Vacant Land, Townhouse, Condo/Co-op]","[280000, 33500, 457000, 17900, 363000, 129000, 298000, 4125, 349990, 10000, 230000, 290000, 25000, 348000, 26000, 15000, 15000, 26000, 34998, 18000, 16500, 18000, 18000, 19900, 14900, 60000, 15500, 152000, 145000, 269000, 195000, 110000, 140000, 10000, 200000, 158000, 259900, 17000, 198000, 30000, 14000, 11000, 278000, 216500, 33500, 13000, 55000, 55000, 585000, 349900, 12500, 394900, 25000, 22900, 14500, 13500, 325000, 19000, 19000, 21500, 28000, 19500, 80000]",2,Fort Myers,51,Fort Myers,Sellstate 5 Star Realty,The Anderson Team - Fort Myers,- Partner Team,The Anderson Team - Fort Myers - Partner Team,4,True,False,/real-estate-agents/the-anderson-team-fort-myers,NaN,True,8.0,True,False,NaN,NaN,NaN,63,33500.0,17950.0,208250.0,4125.0,585000.0,139226.793229,6,4
2,Alabama,21113,True,Kathy Stobb,Kathy,kathy@eventiderealty.com,katherine-stobb,https://ssl.cdn-redfin.com/v594.0.0/images/serviceProvider/person-500x500.jpg,NaN,False,True,True,5.0,36,0,12.0,False,False,[],NaN,NaN,NaN,FL,Senior Agent,"[Cape Coral, Lehigh Acres, FL - Alva & LaBelle]","[Vacant Land, Single Family Residential, Condo/Co-op, Mobile/Manufactured Home, Multi-Family (2-4 Unit), Multi-Family (5+ Unit), Other]","[33000, 215000, 365000, 290000, 50000, 189900, 340000, 211500, 248000, 362500, 102500, 455000, 266000, 150000, 146000, 280000, 120000, 139000, 205500, 167000

In [14]:
for col in df.columns:
    print(col)

state
agentId
partner
name
givenName
email
slug
photoUrl
phoneNumber
adminOnlyPhoneNumber
servesOffers
servesListings
starRating
numReviews
pastYearDealsInRegion
pastYearDeals
veteran
isPremier
languages
complianceText
homeTransactionsLifetime
transactionVolumeLifetime
officeState
jobTitle
primaryServiceRegions
propertyTypes
dealPrices
agentType
businessMarket
businessMarketId
servicing
brokerageName
firstName
lastName
fullName
gender
hasPhotos
noPhoto
profileUrl
photoUrl640x460
profileContactEnabled
totalDealsInPastYear
isActive
isPartnerTeam
customJobTitle
licenseClassTitle
licenseNumber
dealPrices_count
dealPrices_median
dealPrices_q25
dealPrices_q75
dealPrices_min
dealPrices_max
dealPrices_std
num_service_regions
num_property_types


In [15]:
df.shape

(356, 56)

In [16]:
df_clean_cols = [
    # IDs / join
    "agentId", "slug",

    # Display & contact
    "name", "photoUrl", "profileUrl",
    "brokerageName", "jobTitle", "businessMarket", "officeState",
    "email", "phoneNumber",

    # Quality / activity
    "starRating", "numReviews", "pastYearDeals", "pastYearDealsInRegion",
    "homeTransactionsLifetime", "transactionVolumeLifetime",

    # Coverage & specialization
    "primaryServiceRegions", "num_service_regions",
    "propertyTypes", "num_property_types", "businessMarketId",

    # Price stats (engineered)
    "dealPrices_count", "dealPrices_median", "dealPrices_q25", "dealPrices_q75",
    "dealPrices_min", "dealPrices_max", "dealPrices_std",

    # Flags
    "partner", "isPremier", "servesOffers", "servesListings",
    "isActive", "profileContactEnabled",

    # Optional
    "languages"
]
df_clean = df[df_clean_cols].copy()


In [17]:
df_clean.head()

,agentId,slug,name,photoUrl,profileUrl,brokerageName,jobTitle,businessMarket,officeState,email,phoneNumber,starRating,numReviews,pastYearDeals,pastYearDealsInRegion,homeTransactionsLifetime,transactionVolumeLifetime,primaryServiceRegions,num_service_regions,propertyTypes,num_property_types,businessMarketId,dealPrices_count,dealPrices_median,dealPrices_q25,dealPrices_q75,dealPrices_min,dealPrices_max,dealPrices_std,partner,isPremier,servesOffers,servesListings,isActive,profileContactEnabled,languages
0,24746,david-squires,David Squires,https://ssl.cdn-redfin.com/system_files/images/24746/640x460/6_63.jpg,/real-estate-agents/david-squires,Redfin,Senior Agent,Fort Myers,FL,david.squires@redfin.com,(239) 341-5997,4.9,51,19.0,0,117.0,52659458.0,"[N Fort Myers, Lehigh Acres, Fort Myers, Fort Myers Beach, Bonita Springs-Estero, East Fort Myers]",6,"[Single Family Residential, Condo/Co-op, Vacant Land, Mobile/Manufactured Home, Multi-Family (2-4 Unit)]",5,51,86,387500.0,302750.0,531380.0,140000.0,2900000.0,353690.279493,False,True,True,True,True,False,[]
1,37611,the-anderson-team-fort-myers,The Anderson Team - Fort Myers - Partner Team,https://ssl.cdn-redfin.com/system_files/images/37611/500x500/8_58.jpg,/real-estate-agents/the-anderson-team-fort-myers,Sellstate 5 Star Realty,Senior Agent,Fort Myers,FL,partner-team-1742577788@redfintest.com,NaN,5.0,18,8.0,1,NaN,NaN,"[Cape Coral, N Fort Myers, Lehigh Acres, Fort Myers, East Fort Myers, Fort Myers Islands]",6,"[Single Family Residential, Vacant Land, Townhouse, Condo/Co-op]",4,51,63,33500.0,17950.0,208250.0,4125.0,585000.0,139226.793229,True,False,True,True,True,True,[]
2,21113,katherine-stobb,Kathy Stobb,https://ssl.cdn-redfin.com/v594.0.0/images/serviceProvider/person-500x500.jpg,/real-estate-agents/katherine-stobb,Eventide Realty Services LLC,Senior Agent,Fort Myers,FL,kathy@eventiderealty.com,NaN,5.0,36,12.0,0,NaN,NaN,"[Cape Coral, Lehigh Acres, FL - Alva & LaBelle]",3,"[Vacant Land, Single Family Residential, Condo/Co-op, Mobile/Manufactured Home, Multi-Family (2-4 Unit), Multi-Family (5+ Unit), Other]",7,51,125,244000.0,152500.0,335000.0,20000.0,2100000.0,207710.996928,True,False,True,True,True,True,[]
3,46712,jude-paul,Jude Paul,https://ssl.cdn-redfin.com/v594.0.0/images/serviceProvider/person-500x500.jpg,/real-estate-agents/jude-paul,Suncoast Realty Solutions,Senior Agent,Fort Myers,FL,jude.paul@suncoastrs.com,NaN,5.0,11,1.0,0,NaN,NaN,"[Cape Coral, Lehigh Acres, Fort Myers, East Fort Myers]",4,"[Single Family Residential, Other, Multi-Family (2-4 Unit), Condo/Co-op, Townhouse, Vacant Land]",6,51,41,260000.0,210000.0,340000.0,21500.0,674000.0,108284.251100,True,False,True,True,True,True,[]
4,61995,knowledge-base-team,Knowledge Base TEAM - Partner Team,https://ssl.cdn-redfin.com/system_files/images/61995/500x500/8_34.jpg,/real-estate-agents/knowledge-base-team,Knowledge Base Real Estate,Senior Agent,Fort Myers,FL,partner-team-1741383035@redfintest.com,NaN,4.9,174,1.0,0,NaN,NaN,"[Cape Coral, N Fort Myers, Lehigh Acres, Fort Myers, Fort Myers Beach, Bonita Springs-Estero, Ave Maria, East Fort Myers, North Naples]",9,[Condo/Co-op],1,51,1,70000.0,70000.0,70000.0,70000.0,70000.0,0.000000,True,False,True,True,True,True,[]


In [18]:
df_clean.shape

(356, 36)

In [19]:
df_clean.isnull().sum()

agentId                        0
slug                           0
name                           0
photoUrl                       3
profileUrl                     0
brokerageName                101
jobTitle                       0
businessMarket                 0
officeState                   74
email                          0
phoneNumber                  190
starRating                     0
numReviews                     0
pastYearDeals                 89
pastYearDealsInRegion          0
homeTransactionsLifetime     202
transactionVolumeLifetime    200
primaryServiceRegions          0
num_service_regions            0
propertyTypes                  0
num_property_types             0
businessMarketId               0
dealPrices_count               0
dealPrices_median              0
dealPrices_q25                 0
dealPrices_q75                 0
dealPrices_min                 0
dealPrices_max                 0
dealPrices_std                 0
partner                        0
isPremier 

In [20]:
mask = (df_clean['dealPrices_count'] > 0) & (df_clean['homeTransactionsLifetime'].isna())
hits = df_clean.loc[mask]          # rows that match
n = mask.sum()               # how many rows match


In [21]:
n

133

In [22]:
hits.head(3)

,agentId,slug,name,photoUrl,profileUrl,brokerageName,jobTitle,businessMarket,officeState,email,phoneNumber,starRating,numReviews,pastYearDeals,pastYearDealsInRegion,homeTransactionsLifetime,transactionVolumeLifetime,primaryServiceRegions,num_service_regions,propertyTypes,num_property_types,businessMarketId,dealPrices_count,dealPrices_median,dealPrices_q25,dealPrices_q75,dealPrices_min,dealPrices_max,dealPrices_std,partner,isPremier,servesOffers,servesListings,isActive,profileContactEnabled,languages
1,37611,the-anderson-team-fort-myers,The Anderson Team - Fort Myers - Partner Team,https://ssl.cdn-redfin.com/system_files/images/37611/500x500/8_58.jpg,/real-estate-agents/the-anderson-team-fort-myers,Sellstate 5 Star Realty,Senior Agent,Fort Myers,FL,partner-team-1742577788@redfintest.com,NaN,5.0,18,8.0,1,NaN,NaN,"[Cape Coral, N Fort Myers, Lehigh Acres, Fort Myers, East Fort Myers, Fort Myers Islands]",6,"[Single Family Residential, Vacant Land, Townhouse, Condo/Co-op]",4,51,63,33500.0,17950.0,208250.0,4125.0,585000.0,139226.793229,True,False,True,True,True,True,[]
2,21113,katherine-stobb,Kathy Stobb,https://ssl.cdn-redfin.com/v594.0.0/images/serviceProvider/person-500x500.jpg,/real-estate-agents/katherine-stobb,Eventide Realty Services LLC,Senior Agent,Fort Myers,FL,kathy@eventiderealty.com,NaN,5.0,36,12.0,0,NaN,NaN,"[Cape Coral, Lehigh Acres, FL - Alva & LaBelle]",3,"[Vacant Land, Single Family Residential, Condo/Co-op, Mobile/Manufactured Home, Multi-Family (2-4 Unit), Multi-Family (5+ Unit), Other]",7,51,125,244000.0,152500.0,335000.0,20000.0,2100000.0,207710.996928,True,False,True,True,True,True,[]
3,46712,jude-paul,Jude Paul,https://ssl.cdn-redfin.com/v594.0.0/images/serviceProvider/person-500x500.jpg,/real-estate-agents/jude-paul,Suncoast Realty Solutions,Senior Agent,Fort Myers,FL,jude.paul@suncoastrs.com,NaN,5.0,11,1.0,0,NaN,NaN,"[Cape Coral, Lehigh Acres, Fort Myers, East Fort Myers]",4,"[Single Family Residential, Other, Multi-Family (2-4 Unit), Condo/Co-op, Townhouse, Vacant Land]",6,51,41,260000.0,210000.0,340000.0,21500.0,674000.0,108284.251100,True,False,True,True,True,True,[]


In [23]:
df_clean['homeTransactionsLifetime'] = (
    df_clean['homeTransactionsLifetime'].fillna(df_clean['dealPrices_count'])
)


In [24]:
mask = (df_clean['dealPrices_count'] > 0) & (df_clean['transactionVolumeLifetime'].isna())
hits = df_clean.loc[mask]          # rows that match
n = mask.sum()               # how many rows match


In [25]:
n

131

In [26]:
hits.head(3)

,agentId,slug,name,photoUrl,profileUrl,brokerageName,jobTitle,businessMarket,officeState,email,phoneNumber,starRating,numReviews,pastYearDeals,pastYearDealsInRegion,homeTransactionsLifetime,transactionVolumeLifetime,primaryServiceRegions,num_service_regions,propertyTypes,num_property_types,businessMarketId,dealPrices_count,dealPrices_median,dealPrices_q25,dealPrices_q75,dealPrices_min,dealPrices_max,dealPrices_std,partner,isPremier,servesOffers,servesListings,isActive,profileContactEnabled,languages
1,37611,the-anderson-team-fort-myers,The Anderson Team - Fort Myers - Partner Team,https://ssl.cdn-redfin.com/system_files/images/37611/500x500/8_58.jpg,/real-estate-agents/the-anderson-team-fort-myers,Sellstate 5 Star Realty,Senior Agent,Fort Myers,FL,partner-team-1742577788@redfintest.com,NaN,5.0,18,8.0,1,63.0,NaN,"[Cape Coral, N Fort Myers, Lehigh Acres, Fort Myers, East Fort Myers, Fort Myers Islands]",6,"[Single Family Residential, Vacant Land, Townhouse, Condo/Co-op]",4,51,63,33500.0,17950.0,208250.0,4125.0,585000.0,139226.793229,True,False,True,True,True,True,[]
2,21113,katherine-stobb,Kathy Stobb,https://ssl.cdn-redfin.com/v594.0.0/images/serviceProvider/person-500x500.jpg,/real-estate-agents/katherine-stobb,Eventide Realty Services LLC,Senior Agent,Fort Myers,FL,kathy@eventiderealty.com,NaN,5.0,36,12.0,0,125.0,NaN,"[Cape Coral, Lehigh Acres, FL - Alva & LaBelle]",3,"[Vacant Land, Single Family Residential, Condo/Co-op, Mobile/Manufactured Home, Multi-Family (2-4 Unit), Multi-Family (5+ Unit), Other]",7,51,125,244000.0,152500.0,335000.0,20000.0,2100000.0,207710.996928,True,False,True,True,True,True,[]
3,46712,jude-paul,Jude Paul,https://ssl.cdn-redfin.com/v594.0.0/images/serviceProvider/person-500x500.jpg,/real-estate-agents/jude-paul,Suncoast Realty Solutions,Senior Agent,Fort Myers,FL,jude.paul@suncoastrs.com,NaN,5.0,11,1.0,0,41.0,NaN,"[Cape Coral, Lehigh Acres, Fort Myers, East Fort Myers]",4,"[Single Family Residential, Other, Multi-Family (2-4 Unit), Condo/Co-op, Townhouse, Vacant Land]",6,51,41,260000.0,210000.0,340000.0,21500.0,674000.0,108284.251100,True,False,True,True,True,True,[]


In [27]:
def clean_prices(seq):
    if not isinstance(seq, list): 
        return []
    out = []
    for v in seq:
        try:
            x = float(v)
            if x > 0 and math.isfinite(x):
                out.append(x)
        except Exception:
            pass
    return out

# If df_clean still has the raw list column:
df_clean["transactionVolumeLifetime"] = df["dealPrices"].apply(clean_prices).apply(sum)

In [28]:
df_clean["officeState"] = df_clean["officeState"].fillna("Unknown")
df_clean["brokerageName"] = df_clean["brokerageName"].fillna("Unknown")
df_clean["photoUrl"] = df_clean["photoUrl"].fillna("Unknown")
df_clean["phoneNumber"]=df_clean["phoneNumber"].fillna("Unknown")
df_clean["pastYearDeals"]=df_clean["pastYearDeals"].fillna(0).astype(int)

In [29]:
if {"homeTransactionsLifetime","transactionVolumeLifetime"}.issubset(df.columns):
    denom = df_clean["homeTransactionsLifetime"].replace(0, np.nan)
    df_clean["avg_transaction_value"] = (df_clean["transactionVolumeLifetime"] / denom).fillna(0)
else:
    df_clean["avg_transaction_value"] = 0.0

In [30]:
df_clean.isnull().sum()

agentId                      0
slug                         0
name                         0
photoUrl                     0
profileUrl                   0
brokerageName                0
jobTitle                     0
businessMarket               0
officeState                  0
email                        0
phoneNumber                  0
starRating                   0
numReviews                   0
pastYearDeals                0
pastYearDealsInRegion        0
homeTransactionsLifetime     0
transactionVolumeLifetime    0
primaryServiceRegions        0
num_service_regions          0
propertyTypes                0
num_property_types           0
businessMarketId             0
dealPrices_count             0
dealPrices_median            0
dealPrices_q25               0
dealPrices_q75               0
dealPrices_min               0
dealPrices_max               0
dealPrices_std               0
partner                      0
isPremier                    0
servesOffers                 0
servesLi

In [31]:
df_clean.dtypes

agentId                        int64
slug                          object
name                          object
photoUrl                      object
profileUrl                    object
brokerageName                 object
jobTitle                      object
businessMarket                object
officeState                   object
email                         object
phoneNumber                   object
starRating                   float64
numReviews                     int64
pastYearDeals                  int64
pastYearDealsInRegion          int64
homeTransactionsLifetime     float64
transactionVolumeLifetime    float64
primaryServiceRegions         object
num_service_regions            int64
propertyTypes                 object
num_property_types             int64
businessMarketId               int64
dealPrices_count               int64
dealPrices_median            float64
dealPrices_q25               float64
dealPrices_q75               float64
dealPrices_min               float64
d

In [32]:
import pandas as pd
import numpy as np
import plotly.express as px

# ---- start from your df_clean ----
dfv = df_clean.copy()

# type coercions for axes
for c in ["starRating", "numReviews", "pastYearDeals"]:
    if c in dfv.columns:
        dfv[c] = pd.to_numeric(dfv[c], errors="coerce")

# bool normalize
if "isPremier" in dfv.columns:
    dfv["isPremier"] = dfv["isPremier"].map(
        {True: True, False: False, "True": True, "False": False, 1: True, 0: False}
    ).fillna(False)

# add 'state' if missing (fallback to officeState)
if "state" not in dfv.columns and "officeState" in dfv.columns:
    dfv["state"] = dfv["officeState"]

# keep only rows with required axes
dfv = dfv.dropna(subset=["numReviews", "pastYearDeals", "starRating"])

# helper: keep only hover columns that actually exist
def safe_hover(cols):
    return [c for c in cols if c in dfv.columns]

HOVER = safe_hover(["name","brokerageName","state","starRating","isPremier"])

# ---------- Charts ----------


In [33]:
# 1) Scatter: numReviews vs pastYearDeals
fig = px.scatter(
    dfv, x="numReviews", y="pastYearDeals",
    hover_data=HOVER, title="numReviews vs pastYearDeals"
)
fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.update_layout(height=500)
fig.show()

In [34]:
# 2) Scatter with log axes (better for heavy tails)
fig = px.scatter(
    dfv, x="numReviews", y="pastYearDeals",
    hover_data=HOVER, log_x=True, log_y=True,
    title="numReviews vs pastYearDeals (log scales)"
)
fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.update_layout(height=500)
fig.show()


In [35]:
# 3) Scatter: starRating vs pastYearDeals
fig = px.scatter(
    dfv, x="starRating", y="pastYearDeals",
    hover_data=safe_hover(["name","brokerageName","state","numReviews","isPremier"]),
    range_x=[4,5], title="starRating vs pastYearDeals"
)
fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.update_layout(height=500)
fig.show()

In [36]:
# 4) Box plot: pastYearDeals by isPremier
fig = px.box(
    dfv, x="isPremier", y="pastYearDeals",
    points="outliers",
    labels={"isPremier": "isPremier (False/True)"},
    title="Distribution of pastYearDeals by isPremier"
)
fig.update_layout(height=500)
fig.show()

In [37]:
cols = ["starRating", "numReviews", "isPremier", "pastYearDeals"]
d = df_clean[cols].copy()

# ensure dtypes
d["starRating"]     = pd.to_numeric(d["starRating"], errors="coerce")
d["numReviews"]     = pd.to_numeric(d["numReviews"], errors="coerce")
d["pastYearDeals"]  = pd.to_numeric(d["pastYearDeals"], errors="coerce")
d["isPremier"]      = d["isPremier"].astype(bool)

# 1) matrix using Pearson (linear)
pearson_corr = d.assign(isPremier=d["isPremier"].astype(int)).corr(numeric_only=True, method="pearson")
print("pearson_corr")
print(pearson_corr)

# 2) also try rank-based (robust to nonlinearity / heavy tails)
spearman_corr = d.assign(isPremier=d["isPremier"].astype(int)).corr(numeric_only=True, method="spearman")
print("\nspearman_corr")
print(spearman_corr)

# 3) correlation of each feature vs pastYearDeals
print("\n")
print(d.assign(isPremier=d["isPremier"].astype(int)).corrwith(d["pastYearDeals"], numeric_only=True))

# 4) optional: point-biserial for isPremier vs pastYearDeals
from scipy.stats import pointbiserialr
r, p = pointbiserialr(d["isPremier"].astype(int), d["pastYearDeals"])
print("point-biserial (isPremier vs pastYearDeals): r=", r, " p=", p)


pearson_corr
               starRating  numReviews  isPremier  pastYearDeals
starRating       1.000000    0.304398   0.286098       0.436319
numReviews       0.304398    1.000000   0.624877       0.653644
isPremier        0.286098    0.624877   1.000000       0.696059
pastYearDeals    0.436319    0.653644   0.696059       1.000000

spearman_corr
               starRating  numReviews  isPremier  pastYearDeals
starRating       1.000000    0.383162   0.027705       0.326012
numReviews       0.383162    1.000000   0.608683       0.782954
isPremier        0.027705    0.608683   1.000000       0.623688
pastYearDeals    0.326012    0.782954   0.623688       1.000000


starRating       0.436319
numReviews       0.653644
isPremier        0.696059
pastYearDeals    1.000000
dtype: float64
point-biserial (isPremier vs pastYearDeals): r= 0.696058618493514  p= 7.047413564404052e-53


In [38]:
# PUBLIC_COLS = [
#     "agentId","name","photoUrl","businessMarket","brokerageName","jobTitle","officeState",
#     "starRating","numReviews","pastYearDeals","pastYearDealsInRegion",
#     "primaryServiceRegions","propertyTypes","profileUrl","slug","partner","isPremier"
# ]
# agents_public = df[[c for c in PUBLIC_COLS if c in df.columns]].copy()
# agents_public.head(3)

In [39]:
# MODEL_COLS = [
#     # identifiers / meta
#     "agentId","name","email","phoneNumber","businessMarket","brokerageName","jobTitle","officeState",
#     # base signals used by FeatureExtractor & Baseline scorer
#     "starRating","numReviews","pastYearDeals","pastYearDealsInRegion",
#     "homeTransactionsLifetime","transactionVolumeLifetime",
#     # engineered price stats
#     "dealPrices_median","dealPrices_q25","dealPrices_q75","dealPrices_min","dealPrices_max","dealPrices_std","dealPrices_count",
#     # list features + counts
#     "primaryServiceRegions","propertyTypes","num_service_regions","num_property_types",
#     # flags consumed by baseline
#     "partner","isPremier","servesOffers","servesListings","isActive","profileContactEnabled",
#     # convenience
#     "avg_transaction_value","photoUrl","profileUrl","slug"
# ]
# agents_ml_features = df[[c for c in MODEL_COLS if c in df.columns]].copy()
# agents_ml_features.head(3)


In [40]:
from pathlib import Path

out = Path("/kaggle/working/agents_clean.csv")
df_clean.to_csv(out, index=False, encoding="utf-8")

In [41]:
# =============================
# Agent Insights from df_clean (Plotly)
# =============================
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ---------- 0) Prep ----------
df = df_clean.copy()


# numeric coercions (safe)
for c in ["starRating","numReviews","pastYearDeals","dealPrices_median",
          "dealPrices_count","num_service_regions","avg_transaction_value"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# boolean normalize
if "isPremier" in df.columns:
    df["isPremier"] = df["isPremier"].map({True:True, False:False, "True":True, "False":False, 1:True, 0:False}).fillna(False)

# working view (need pastYearDeals for most plots)
dfv = df.dropna(subset=["pastYearDeals"]).copy()

# helper: only include existing hover columns (avoids ValueErrors)
def safe_hover(dframe, cols):
    return [c for c in cols if c in dframe.columns]



In [42]:
# ============================================================
# 1) Feature Correlation with Deal Volume (bar)
# ============================================================
features = {
    "isPremier": dfv["isPremier"].astype(int) if "isPremier" in dfv else pd.Series(index=dfv.index, dtype=float),
    "numReviews": dfv["numReviews"] if "numReviews" in dfv else pd.Series(index=dfv.index, dtype=float),
    "starRating": dfv["starRating"] if "starRating" in dfv else pd.Series(index=dfv.index, dtype=float),
    "serviceRegions": dfv["num_service_regions"] if "num_service_regions" in dfv else pd.Series(index=dfv.index, dtype=float),
}
rows = []
for name, s in features.items():
    rows.append({"feature": name, "correlation": s.corr(dfv["pastYearDeals"])})
corr_plot = pd.DataFrame(rows).fillna(0).sort_values("correlation", ascending=False)

fig = px.bar(
    corr_plot, x="feature", y="correlation",
    title="Feature Correlation with Deal Volume",
    hover_data=["correlation"]
)
fig.update_traces(hovertemplate="%{x}: %{y:.2f}")
fig.show()


In [43]:
# ============================================================
# 2) Premium vs Standard Agent Performance (two panels)
#    Left: Avg deals/year; Right: Median price
# ============================================================
if "isPremier" in dfv and "dealPrices_median" in dfv:
    g = dfv.groupby("isPremier", dropna=False)
    avg_deals = g["pastYearDeals"].mean().rename("avg_deals").reset_index()
    med_price = g["dealPrices_median"].median().rename("median_price").reset_index()

    fig = make_subplots(rows=1, cols=2, subplot_titles=("Avg Deals / Year", "Median Price"))
    fig.add_trace(go.Bar(
        x=avg_deals["isPremier"].astype(str), y=avg_deals["avg_deals"],
        name="Avg Deals / Year", hovertemplate="isPremier=%{x}<br>Avg Deals=%{y:.1f}"
    ), row=1, col=1)
    fig.add_trace(go.Bar(
        x=med_price["isPremier"].astype(str), y=med_price["median_price"],
        name="Median Price", hovertemplate="isPremier=%{x}<br>Median Price=$%{y:,.0f}"
    ), row=1, col=2)
    fig.update_layout(title="Premium vs Standard Agent Performance", showlegend=False)
    fig.show()

In [44]:
# ============================================================
# 3) Agent Archetype Distribution (pie + summary)
#    Simple rules using volume/price quantiles
# ============================================================
if "dealPrices_median" in dfv:
    qv25, qv75 = dfv["pastYearDeals"].quantile([0.25, 0.75])
    qp75 = dfv["dealPrices_median"].quantile(0.75)

    def archetype(row):
        v, p, r = row["pastYearDeals"], row["dealPrices_median"], row.get("numReviews", np.nan)
        if pd.notna(v) and v >= qv75: return "High-Volume Specialists"
        if pd.notna(p) and p >= qp75: return "Luxury Specialists"
        if (pd.notna(v) and v <= qv25) and (pd.notna(r) and r <= dfv["numReviews"].quantile(0.25)): return "New Agents"
        return "Generalists"

    dfv["archetype"] = dfv.apply(archetype, axis=1)

    arch_cnt = dfv["archetype"].value_counts(dropna=False).rename_axis("archetype").reset_index(name="count")
    fig = px.pie(
        arch_cnt, names="archetype", values="count",
        title="Agent Archetype Distribution"
    )
    # textinfo/texttemplate for compact labels
    fig.update_traces(textinfo="label+percent")
    fig.show()

    # summary table
    arch_stats = (dfv.groupby("archetype")
                    .agg(avg_deals=("pastYearDeals","mean"),
                         avg_price=("dealPrices_median","median"))
                    .reset_index()
                    .sort_values("avg_deals", ascending=False))
    display(arch_stats)

,archetype,avg_deals,avg_price
1,High-Volume Specialists,33.516129,450000.0
2,Luxury Specialists,7.211538,635375.0
0,Generalists,6.027778,301000.0
3,New Agents,0.000000,0.0


In [45]:
# ============================================================
# 4) Experience Tiers & Performance (dual-axis grouped bars)
# ============================================================
bins = [0, 10, 50, 100, np.inf]
labels = ["Low (0-10 deals)","Mid (11-50)","Upper (51-100)","Veteran (100+ deals)"]
dfv["experience_tier"] = pd.cut(dfv["pastYearDeals"], bins=bins, labels=labels, right=True, include_lowest=True)

if "dealPrices_median" in dfv:
    tier_counts = dfv["experience_tier"].value_counts().rename_axis("experience_tier").reset_index(name="agent_count")
    tier_price  = dfv.groupby("experience_tier")["dealPrices_median"].median().rename("median_price").reset_index()
    tier = (tier_counts.merge(tier_price, on="experience_tier", how="left")
                      .set_index("experience_tier").reindex(labels).reset_index())

    fig = make_subplots(specs=[[{"secondary_y": True}]])  # secondary axis for price
    fig.add_trace(go.Bar(x=tier["experience_tier"], y=tier["agent_count"], name="Agent Count"), secondary_y=False)
    fig.add_trace(go.Bar(x=tier["experience_tier"], y=tier["median_price"], name="Median Price"), secondary_y=True)
    fig.update_layout(title="Experience Tiers & Performance", barmode="group")
    fig.update_yaxes(title_text="Agent Count", secondary_y=False)
    fig.update_yaxes(title_text="Median Price ($)", secondary_y=True)
    fig.show()

In [46]:

# ============================================================
# 5) State-Level Performance (dual-axis grouped bars)
# ============================================================
if "officeState" in dfv and "dealPrices_median" in dfv:
    top_states = dfv["officeState"].value_counts().head(5).index.tolist()
    ds = dfv[dfv["officeState"].isin(top_states)]
    st_deals = ds.groupby("officeState")["pastYearDeals"].mean().rename("avg_deals").reset_index()
    st_price = ds.groupby("officeState")["dealPrices_median"].median().rename("median_price").reset_index()
    st = st_deals.merge(st_price, on="officeState", how="inner")

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Bar(x=st["officeState"], y=st["avg_deals"], name="Avg Deals/Year"), secondary_y=False)
    fig.add_trace(go.Bar(x=st["officeState"], y=st["median_price"], name="Median Price"), secondary_y=True)
    fig.update_layout(title="State-Level Performance Metrics", barmode="group")
    fig.update_yaxes(title_text="Avg Deals/Year", secondary_y=False)
    fig.update_yaxes(title_text="Median Price ($)", secondary_y=True)
    fig.show()

In [47]:
# ============================================================
# 6) Property-Type Specialization (dual-axis grouped bars)
# ============================================================
if "propertyTypes" in dfv and "dealPrices_median" in dfv:
    def as_list(v):
        if isinstance(v, list): return v
        if pd.isna(v) or v is None: return []
        return [v]

    tmp = dfv.copy()
    tmp["propertyTypes"] = tmp["propertyTypes"].apply(as_list)
    ex  = tmp.explode("propertyTypes").dropna(subset=["propertyTypes"])

    by_type = (ex.groupby("propertyTypes")
                 .agg(agent_count=("agentId","nunique"),
                      median_price=("dealPrices_median","median"))
                 .reset_index()
                 .sort_values("agent_count", ascending=False))

    top_types = by_type.head(6)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Bar(x=top_types["propertyTypes"], y=top_types["agent_count"], name="Agent Count"), secondary_y=False)
    fig.add_trace(go.Bar(x=top_types["propertyTypes"], y=top_types["median_price"], name="Median Price"), secondary_y=True)
    fig.update_layout(title="Property Type Specialization", barmode="group")
    fig.update_yaxes(title_text="Agent Count", secondary_y=False)
    fig.update_yaxes(title_text="Median Price ($)", secondary_y=True)
    fig.show()